In [64]:
import pandas as pd

df = pd.read_csv("student-scores.csv")
df.head()

,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,False,2,False,47,Doctor,90,86,96,100,90,88,90
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,False,5,False,3,Artist,71,74,88,80,89,63,86
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,False,5,False,10,Unknown,84,77,65,65,80,74,76


In [65]:
df.drop(columns=['id','first_name','last_name','email'],axis=1,inplace=True)

In [66]:
df.shape

(2000, 13)

In [67]:
df["total_score"] = df["math_score"] + df["history_score"] + df["physics_score"] + df["chemistry_score"] + df["biology_score"] + df["english_score"] + df["geography_score"]
df["average_score"] = df["total_score"] / 7
df.head()

,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,total_score,average_score
0,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87,574,82.000000
1,female,False,2,False,47,Doctor,90,86,96,100,90,88,90,640,91.428571
2,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94,605,86.428571
3,female,False,5,False,3,Artist,71,74,88,80,89,63,86,551,78.714286
4,male,False,5,False,10,Unknown,84,77,65,65,80,74,76,521,74.428571


In [68]:
from sklearn.preprocessing import LabelEncoder

# Assuming your dataframe is named 'df'

# List of categorical columns to encode
categorical_columns = ['gender', 'part_time_job', 'extracurricular_activities', 'career_aspiration']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Check the encoded data
df.head()


,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,total_score,average_score
0,1,0,3,0,27,9,73,81,93,97,63,80,87,574,82.000000
1,0,0,2,0,47,6,90,86,96,100,90,88,90,640,91.428571
2,0,0,9,1,13,8,81,97,95,96,65,77,94,605,86.428571
3,0,0,5,0,3,1,71,74,88,80,89,63,86,551,78.714286
4,1,0,5,0,10,15,84,77,65,65,80,74,76,521,74.428571


In [69]:
# Assuming your dataframe is named 'df' and you want to get unique values for the column 'career_aspiration'

unique_counts = df['career_aspiration'].value_counts()

# Display the unique values and their counts
print(unique_counts)


career_aspiration
12    315
3     309
15    223
2     169
9     138
0     126
6     119
10     83
13     73
4      68
1      67
7      63
8      61
14     59
5      56
11     39
16     32
Name: count, dtype: int64


In [70]:
from imblearn.combine import SMOTETomek
import pandas as pd

# Assuming 'df' is your DataFrame with features and the target variable
X = df.drop('career_aspiration', axis=1)  # Features (all columns except target)
y = df['career_aspiration']  # Target variable

# Initialize SMOTE + Tomek with a fixed random seed for reproducibility
random_seed = 42  # You can set this to any integer value
smote_tomek = SMOTETomek(random_state=random_seed)

# Fit and resample the dataset
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)

# Update 'df' with the balanced data
df = pd.DataFrame(X_resampled, columns=X.columns)
df['career_aspiration'] = y_resampled

# Check the distribution of the target variable after balancing
print(df['career_aspiration'].value_counts())


career_aspiration
5     314
14    314
7     314
16    314
13    313
11    313
4     313
8     312
6     312
1     311
10    311
9     310
0     310
3     308
2     307
15    299
12    292
Name: count, dtype: int64


In [71]:
y_resampled.shape

(5267,)

In [72]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1600, 14), (1600,), (400, 14), (400,))

Feature Scaling

In [74]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [75]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Support Vector Machine': SVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    print("="*50)
    print("Model:", name)
    # Train the model
    model.fit(X_train_scaled, y_train)

    # Predict on test set
    y_pred = model.predict(X_test_scaled)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:\n", classification_rep)
    print("Confusion Matrix:\n", conf_matrix)


Model: Logistic Regression
Accuracy: 0.4475
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.46      0.55        24
           1       0.43      0.21      0.29        14
           2       0.23      0.17      0.19        30
           3       0.86      0.94      0.90        66
           4       0.31      0.42      0.36        12
           5       0.00      0.00      0.00        16
           6       0.47      0.72      0.57        25
           7       0.43      0.75      0.55        12
           8       0.00      0.00      0.00        13
           9       0.43      0.43      0.43        28
          10       0.18      0.21      0.19        14
          11       0.33      0.10      0.15        10
          12       0.34      0.55      0.42        53
          13       0.00      0.00      0.00        10
          14       0.25      0.12      0.17        16
          15       0.33      0.35      0.34        52
          16 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Accuracy: 0.4350
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.38      0.50        24
           1       0.33      0.21      0.26        14
           2       0.13      0.13      0.13        30
           3       0.84      0.94      0.89        66
           4       0.50      0.08      0.14        12
           5       0.00      0.00      0.00        16
           6       0.47      0.84      0.60        25
           7       0.53      0.67      0.59        12
           8       0.00      0.00      0.00        13
           9       0.47      0.50      0.48        28
          10       0.15      0.21      0.18        14
          11       0.00      0.00      0.00        10
          12       0.33      0.70      0.45        53
          13       0.00      0.00      0.00        10
          14       0.00      0.00      0.00        16
          15       0.29      0.23      0.26        52
          16       0.00      0.00      0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/e

Accuracy: 0.1650
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.00      0.00      0.00        14
           2       0.00      0.00      0.00        30
           3       0.00      0.00      0.00        66
           4       0.00      0.00      0.00        12
           5       0.00      0.00      0.00        16
           6       0.00      0.00      0.00        25
           7       0.00      0.00      0.00        12
           8       0.00      0.00      0.00        13
           9       0.00      0.00      0.00        28
          10       0.06      0.50      0.11        14
          11       0.00      0.00      0.00        10
          12       0.20      1.00      0.34        53
          13       0.00      0.00      0.00        10
          14       0.00      0.00      0.00        16
          15       0.23      0.12      0.15        52
          16       0.00      0.00      0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model= RandomForestClassifier(random_state=42)

model.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = model_r.predict(X_test_scaled)

# Calculate metrics
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Report: ", classification_report(y_test, y_pred))
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))

Accuracy:  0.8525
Report:                precision    recall  f1-score   support

           0       0.95      0.83      0.89        24
           1       0.78      1.00      0.88        14
           2       0.83      0.80      0.81        30
           3       1.00      0.95      0.98        66
           4       0.57      1.00      0.73        12
           5       0.81      0.81      0.81        16
           6       0.83      1.00      0.91        25
           7       0.63      1.00      0.77        12
           8       0.87      1.00      0.93        13
           9       0.81      0.89      0.85        28
          10       0.93      0.93      0.93        14
          11       0.91      1.00      0.95        10
          12       0.91      0.74      0.81        53
          13       0.55      0.60      0.57        10
          14       0.94      0.94      0.94        16
          15       0.97      0.62      0.75        52
          16       0.56      1.00      0.71         5


In [78]:
# test 1
print("Actual Label :", y_test.iloc[10])  # Print the actual label for the 10th data point in the test set
print("Model Prediction :", model.predict(X_test_scaled[10].reshape(1,-1))[0])  # Print the model's prediction for the 10th data point (reshaped to 2D)

# Check if the actual label matches the predicted label
if y_test.iloc[10] == model.predict(X_test_scaled[10].reshape(1,-1))[0]:
    print("Wow! Model doing well.....")  # Print a positive message if the prediction is correct
else:
    print("not sure......")  # Print a less confident message if the prediction is incorrect

Actual Label : 15
Model Prediction : 12
not sure......


In [79]:
# test 2
print("Actual Label :", y_test.iloc[90])  # Print the actual label for the 10th data point in the test set
print("Model Prediction :", model.predict(X_test_scaled[90].reshape(1,-1))[0])  # Print the model's prediction for the 10th data point (reshaped to 2D)

# Check if the actual label matches the predicted label
if y_test.iloc[10] == model.predict(X_test_scaled[10].reshape(1,-1))[0]:
    print("Wow! Model doing well.....")  # Print a positive message if the prediction is correct
else:
    print("not sure......")  # Print a less confident message if the prediction is incorrect

Actual Label : 12
Model Prediction : 12
not sure......


In [80]:
# test 3
print("Actual Label :", y_test.iloc[69])  # Print the actual label for the 10th data point in the test set
print("Model Prediction :", model.predict(X_test_scaled[69].reshape(1,-1))[0])  # Print the model's prediction for the 10th data point (reshaped to 2D)

# Check if the actual label matches the predicted label
if y_test.iloc[10] == model.predict(X_test_scaled[10].reshape(1,-1))[0]:
    print("Wow! Model doing well.....")  # Print a positive message if the prediction is correct
else:
    print("not sure......")  # Print a less confident message if the prediction is incorrect

Actual Label : 3
Model Prediction : 3
not sure......


In [81]:
!mkdir Models

mkdir: cannot create directory ‘Models’: File exists


In [82]:
import pickle

# Save the scaler and model
pickle.dump(scaler, open("Models/scaler.pkl", 'wb'))
pickle.dump(model, open("Models/model.pkl", 'wb'))

In [83]:
# Load the scaler and model
scaler = pickle.load(open("Models/scaler.pkl", 'rb'))
model = pickle.load(open("Models/model.pkl", 'rb'))

In [84]:
import pickle
import numpy as np

# Load the scaler, model, and class names
scaler = pickle.load(open("Models/scaler.pkl", 'rb'))
model = pickle.load(open("Models/model.pkl", 'rb'))
class_names = ['Lawyer', 'Doctor', 'Government Officer', 'Artist', 'Unknown',
               'Software Engineer', 'Teacher', 'Business Owner', 'Scientist',
               'Banker', 'Writer', 'Accountant', 'Designer',
               'Construction Engineer', 'Game Developer', 'Stock Investor',
               'Real Estate Developer']

def Recommendations(gender, part_time_job, absence_days, extracurricular_activities,
                    weekly_self_study_hours, math_score, history_score, physics_score,
                    chemistry_score, biology_score, english_score, geography_score,
                    total_score, average_score):

    try:
        # Validate and encode categorical variables
        if gender.lower() not in ['male', 'female']:
            raise ValueError("Gender should be 'male' or 'female'")

        # Encoding: male = 1, female = 0
        gender_encoded = 1 if gender.lower() == 'male' else 0

        # Encoding: part_time_job: False = 0, True = 1
        part_time_job_encoded = 1 if part_time_job else 0

        # Encoding: extracurricular_activities: False = 0, True = 1
        extracurricular_activities_encoded = 1 if extracurricular_activities else 0

        # Create feature array
        feature_array = np.array([[gender_encoded, part_time_job_encoded, absence_days, extracurricular_activities_encoded,
                                   weekly_self_study_hours, math_score, history_score, physics_score,
                                   chemistry_score, biology_score, english_score, geography_score, total_score, average_score]])

        # Scale features
        scaled_features = scaler.transform(feature_array)

        # Predict using the model
        probabilities = model.predict_proba(scaled_features)

        # Get top five predicted classes along with their probabilities
        top_classes_idx = np.argsort(-probabilities[0])[:5]
        top_classes_names_probs = [(class_names[idx], probabilities[0][idx]) for idx in top_classes_idx]

        return top_classes_names_probs

    except Exception as e:
        return f"An error occurred: {str(e)}"


In [85]:
# Example usage 1
final_recommendations = Recommendations(
    gender='female',
    part_time_job=False,
    absence_days=2,
    extracurricular_activities=False,
    weekly_self_study_hours=7,
    math_score=65,
    history_score=60,
    physics_score=97,
    chemistry_score=94,
    biology_score=71,
    english_score=81,
    geography_score=66,
    total_score=534,
    average_score=76.285714
)

print("Top recommended studies with probabilities:")
print("="*50)
for class_name, probability in final_recommendations:
    print(f"{class_name} with probability {probability:.2f}")

Top recommended studies with probabilities:
Game Developer with probability 0.74
Stock Investor with probability 0.15
Scientist with probability 0.05
Writer with probability 0.02
Business Owner with probability 0.01


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [86]:
# Example usage 2
final_recommendations = Recommendations(
    gender='male',
    part_time_job=True,
    absence_days=5,
    extracurricular_activities=True,
    weekly_self_study_hours=10,
    math_score=85,
    history_score=78,
    physics_score=88,
    chemistry_score=90,
    biology_score=75,
    english_score=80,
    geography_score=70,
    total_score=529,
    average_score=75.571429
)

print("Top recommended studies with probabilities:")
print("="*50)
for class_name, probability in final_recommendations:
    print(f"{class_name} with probability {probability:.2f}")

Top recommended studies with probabilities:
Government Officer with probability 0.19
Stock Investor with probability 0.16
Designer with probability 0.13
Writer with probability 0.12
Scientist with probability 0.09


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [87]:
# sklear version in pychar production
import sklearn
print(sklearn.__version__)
# in pycharm env install
# pip install scikit-learn==1.3.2

1.5.2
